### trading view

In [3]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd

# หากไม่มีบัญชี TradingView
tv = TvDatafeed()

# ดึงข้อมูลราคาหุ้น TSLA จาก NASDAQ ในกรอบเวลา 1 ชั่วโมง (1H)
data = tv.get_hist(symbol='BTCUSDT', exchange='BINANCE', interval=Interval.in_1_hour, n_bars=5000)
# data = tv.get_hist(symbol='AMD', exchange='NASDAQ', interval=Interval.in_daily, n_bars=5000)
# data = tv.get_hist(symbol='AMD', exchange='NASDAQ', interval=Interval.in_15_minute, n_bars=5000)

# ตรวจสอบว่ามีข้อมูลหรือไม่
if data is not None and not data.empty:
    # แปลง index เป็นคอลัมน์ Date
    data.reset_index(inplace=True)
    data.rename(columns={'datetime': 'Date', 'close': 'Close', 'high': 'High', 'low': 'Low'}, inplace=True)  # เปลี่ยนชื่อเป็น Date
    
    # บันทึกข้อมูลลงไฟล์ CSV
    filename = "BTC_1K.csv"
    data.to_csv(filename, index=False)  # index=False เพราะเราเก็บ Date ไว้แล้ว
    print(f"Data saved to {filename}")
else:
    print("No data retrieved")


Data saved to BTC_1K.csv


### MT5

In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta

# ตั้งค่าชื่อสินทรัพย์และกรอบเวลา
symbol = "BTCUSD"
timeframe = mt5.TIMEFRAME_H4 # กรอบเวลา 1 วัน

# กำหนดช่วงวันที่
start_date = datetime(2000, 10, 28)  # วันที่ 5 ก.พ. 2024
end_date = datetime(2025, 2, 14)  # วันที่ 18 ก.พ. 2025

# เริ่มต้นการเชื่อมต่อกับ MT5
if not mt5.initialize():
    print("Failed to connect to MT5")
    exit()

# ดึงข้อมูลราคาตามช่วงวันที่
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)

# ปิดการเชื่อมต่อกับ MT5
mt5.shutdown()

# แปลงข้อมูลเป็น DataFrame
if rates is not None and len(rates) > 0:
    df = pd.DataFrame(rates).rename(columns={'time': 'Date', 'close': 'close', 'high': 'high', 'low': 'low', 'open': 'open','tick_volume':'volume'})
    
    # แปลง timestamp เป็น datetime และเพิ่มเวลาไทย (UTC+7)
    df['Date'] = pd.to_datetime(df['Date'], unit='s') 

    # ตั้งค่า Date เป็น index เพื่อให้เป็น DatetimeIndex
    df.set_index('Date', inplace=True)

    # บันทึกลงไฟล์ CSV
    filename = f"BTC_MT5_4H.csv"
    df.to_csv(filename)
    
    print(f"Data saved to {filename}")
else:
    print("No data retrieved")


Data saved to BTC_MT5_4H.csv
